# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf
from object_detection.protos import model_pb2
import time
from IPython.display import clear_output



if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [5]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [6]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [7]:
# What model to download.
MODEL_NAME = 'prod_faster_rcnn_resnet50_coco'
# MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph_new.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
test_image_path = "/home/wc-gpu/MasterThesis/session_data/122923_testing"
# test_image_path = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled/122923_testing"
TEST_IMAGE_PATHS = sorted([os.path.join(test_image_path, f) for f in os.listdir(test_image_path) if f.endswith("jpg")])


# TEST_IMAGE_PATHS = [TEST_IMAGE_PATHS[0], TEST_IMAGE_PATHS[0]]


# Size, in inches, of the output images.
IMAGE_SIZE = (18, 12)

SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
# with tf.Graph().as_default():
#   od_graph_def = tf.GraphDef()
#   with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
#     serialized_graph = fid.read()
#     od_graph_def.ParseFromString(serialized_graph)
#     tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [9]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.5

In [12]:
def run_inference_for_single_image(image):
    with tf.Session(config=tf_config) as sess:

      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = [(op.name, output.name, output.shape) for op in ops for output in op.outputs]
        with open("bla_log.log","w+") as f:
            f.write("\n".join([str(a) for a in all_tensor_names]))
        tensor_dict = {}
    #     for key in [
    #       'num_detections', 'detection_boxes', 'detection_scores',
    #       'detection_classes', 'detection_masks'
    #     ]:
    #         tensor_name = key + ':0'
    #         if tensor_name in all_tensor_names:
    #             tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
    #               tensor_name)
    #     if 'detection_masks' in tensor_dict:
    #         # The following processing is only for single image
    #         detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
    #         detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
    #         # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
    #         real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
    #         detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
    #         detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
    #         detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
    #             detection_masks, detection_boxes, image.shape[0], image.shape[1])
    #         detection_masks_reframed = tf.cast(
    #             tf.greater(detection_masks_reframed, 0.5), tf.uint8)
    #         # Follow the convention by adding back the batch dimension
    #         tensor_dict['detection_masks'] = tf.expand_dims(
    #             detection_masks_reframed, 0)
        features = tf.get_default_graph().get_tensor_by_name(
            'FirstStageFeatureExtractor/resnet_v1_50/resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')

        tensor_dict['features'] = tf.expand_dims(features, 0)



        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

        # Run inference
        output_dict = sess.run(tensor_dict, 
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

        # all outputs are float32 numpy arrays, so convert types as appropriate
    #     output_dict['num_detections'] = int(output_dict['num_detections'][0])
    #     output_dict['detection_classes'] = output_dict[
    #       'detection_classes'][0].astype(np.uint8)
    #     output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    #     output_dict['detection_scores'] = output_dict['detection_scores'][0]
    #     if 'detection_masks' in output_dict:
    #         output_dict['detection_masks'] = output_dict['detection_masks'][0]

        return output_dict

In [8]:
visualize_debug = False

features = None
total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        
        if os.path.exists(os.path.join(session_path, "annotations")):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
            print (session, len(images))
    except:
        continue


164413 55
164398 56
164384 82
164378 45
164377 50
164375 45
164372 46
164349 54
164328 47
164325 63
164313 57
164305 79
164303 80
164280 48
164265 45
164244 46
164233 75
164230 56
164224 58
164216 115
164204 83
164194 57
164172 45
164168 56
164155 60
164154 86
164142 57
164141 46
164140 54
164136 50
164135 47
164122 62
164117 48
164102 78
164098 114
164085 55
164079 51
164065 93
164055 49
164051 45
164050 56
164043 67
164042 47
164031 50
164030 61
164025 47
164022 47
164020 49
164017 66
164004 56
164002 54
163995 70
163989 60
163952 51
163935 61
163927 52
163922 49
163918 48
163915 48
163908 53
163894 50
163893 49
163888 55
163882 46
163873 49
163871 55
163868 71
163866 59
163856 62
163855 63
163851 56
163844 61
163834 89
163832 48
163830 65
163822 128
163812 55
163801 45
163797 57
163795 61
163791 75
163786 59
163767 60
163761 50
163759 63
162783 53
162782 66
162781 68
162778 59
162773 79
162772 45
162769 46
162766 84
162761 133
162758 69
162746 57
162744 78
162743 47
162741 96
162739

In [13]:
progress = 0

processed = 0
time_avg = 0

outputs = []

log= open("feature_extraction_new.log","w+")

with tf.Graph().as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


    for session in sorted(actual_sessions, reverse=True):
        session_path = os.path.join(SESSIONS_PATH, session)

        images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
        for image_fname in sorted(images):
            try:
                t0 = time.time()
                has_processed = False
                features_path = os.path.join(session_path, "features")
                if not os.path.exists(os.path.join(features_path, image_fname+".feature.prod.new.better.small")):

                    image_path = os.path.join(session_path, image_fname)

                    image = Image.open(image_path)
                    image_np = load_image_into_numpy_array(image)

                    output_dict = run_inference_for_single_image(image_np)
                    features = output_dict["features"][0]
                    break
                    features_path = os.path.join(session_path, "features")
                    if not os.path.exists(features_path):
                        os.makedirs(features_path)

                    with open(os.path.join(features_path, image_fname+".feature.prod.new.better.small"), "wb") as f:
                        np.save(f, features)

                    total_time = time_avg * processed
                    processed += 1
                    has_processed = True



                if visualize_debug:
                    outputs.append(output_dict)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                      image_np,
                      output_dict['detection_boxes'],
                      output_dict['detection_classes'],
                      output_dict['detection_scores'],
                      category_index,
                      instance_masks=output_dict.get('detection_masks'),
                      use_normalized_coordinates=True,
                      line_thickness=5,
                      min_score_thresh=.3)
                    plt.figure(figsize=IMAGE_SIZE)
                    plt.imshow(image_np)


                    slc = features[0][:,:,5]
                    slc *= 255.0/slc.max()  

                    img = Image.fromarray(slc)
                    img = img.convert('RGB')

                    plt.figure(figsize=IMAGE_SIZE)
                    plt.imshow(img)

            except Exception as ex:
                print ("IMAGE ", image_fname)
                print (str(ex))


            t1 = time.time()

            total = t1-t0

            if has_processed:
                time_avg = (total_time + total) / processed


            log_text = " ".join(["progress ", session, ":", str(progress / total_images), 
                                 str(progress), str(total_images)])
            log_text += " | "

            log_text += "".join(["process time ", str(total)])
            log_text += " | "
            log_text += "".join([" time passed: ", str(time_avg*processed / 3600) ," h, remaining: ", str(time_avg* (total_images - progress) / 3600), " h"])
            
            
            print (log_text, end="                                       \r")
            log.write(log_text + "\r\n")

            progress += 1

log.close()

In [ ]:
processed

In [ ]:
progress